<a href="https://colab.research.google.com/github/yeji210park/Aespython-Ajou-Univ.-Team4/blob/master/Generate_images_by_using_RBM_on_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'digit-recognizer:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3004%2F861823%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T052839Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dab3a13b1ef1a813bcfbaf5c42babfdcb79741bcee371ff0be54352f40786e3593113eb49615bce19db155c042a019fb9c40fb62efd031188264780764553011780cac18781b76a3289e6d9ebf8d94affc1bc3d23a988c72eed699bafe66d2c1420a2a3ec1dc792a3f037ad909eccbe65456794e2a686de2053148d2430ca6e8f56fa722411f571d6c97bdf5702d77fb88a2abb0b5ff48859bfff5dad29175a93efd77dcdfb3448d0b0579bbf2e8d43e8e872b8e4a8aaa6f9086ad239c52e89d6780621ffed651f23d46ca5214027ecc3fb3f99803dccd3fa0c71ba466addf65376f7aa4ad6182fd6be10d2cf88ae2ef417a0132bac50adadc9d97750d85e8b2d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 16054568 bytes downloaded
Downloaded and uncompressed: digit-recognizer
Data source import complete.


# <p style="padding:10px;background-color:#9c8497 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Generate images by using RBM on MNIST</p>

<div style="border-radius:10px;border:#63445d solid;padding: 15px;background-color:white;font-size:110%;text-align:left">

<h3 align="left"><font color=#63445d>What are Boltzmann Machines?</font></h3>
    

    
It is a network of neurons in which all the neurons are connected to each other. In this machine, there are two layers named visible layer or input layer and hidden layer. The visible layer is denoted as v and the hidden layer is denoted as the h. In Boltzmann machine, there is no output layer. Boltzmann machines are random and generative neural networks capable of learning internal representations and are able to represent and (given enough time) solve tough combinatoric problems.

The Boltzmann distribution (also known as Gibbs Distribution) which is an integral part of Statistical Mechanics and also explain the impact of parameters like Entropy and Temperature on the Quantum States in Thermodynamics. Due to this, it is also known as Energy-Based Models (EBM). It was invented in 1985 by Geoffrey Hinton, then a Professor at Carnegie Mellon University, and Terry Sejnowski, then a Professor at Johns Hopkins University.

![image.png](attachment:image.png)

<div style="border-radius:10px;border:#63445d solid;padding: 15px;background-color:white;font-size:110%;text-align:left">
    
<h3 align="left"><font color=#63445d>What are Restricted Boltzmann Machines (RBM)?</font></h3>
    
RBM is a Stochastic Neural Network which means that each neuron will have some random behavior when activated. There are two other layers of bias units (hidden bias and visible bias) in an RBM. This is what makes RBMs different from autoencoders. The hidden bias RBM produces the activation on the forward pass and the visible bias helps RBM to reconstruct the input during a backward pass. The reconstructed input is always different from the actual input as there are no connections among the visible units and therefore, no way of transferring information among themselves.

# <p style="padding:10px;background-color:#9c8497 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Imort Libraries</p>

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

# <p style="padding:10px;background-color:#9c8497 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Loading Data</p>

<div style="border-radius:10px;border:#63445d solid;padding: 15px;background-color:white;font-size:110%;text-align:left">

<h3 align="left"><font color=#63445d>about dataset</font></h3>
    
The MNIST dataset is one of the most well studied datasets in the computer vision and machine learning literature. In many cases, it’s a benchmark, a standard to which some machine learning algorithms are ranked against.
The goal of this dataset is to correctly classify the handwritten digits 0-9. We are not going to utilize the entire dataset (which consists of 60,000 training images and 10,000 testing images), instead we are going to utilize a small sample (3,000 for training, 2,000 for testing). The data points are approximately uniformly distributed per digit, so no substantial class label imbalance exists.
Each feature vector is 784-dim, corresponding to the 28 x 28 grayscale pixel intensities of the image. These grayscale pixel intensities are unsigned integers, falling into the range [0, 255].
All digits are placed on a black background, with the foreground being white and shades of gray.


In [ ]:
TRAIN_PATH = '/kaggle/input/digit-recognizer/train.csv'
TEST_PATH = '/kaggle/input/digit-recognizer/test.csv'
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [ ]:
X = (train.iloc[:,1:].values).astype('float32').reshape(-1, 28*28)
y = train.iloc[:,0].values.astype('int32')
X_test = test.values.astype('float32').reshape(-1, 28*28)

In [ ]:
# EXTREMELY IMPORTANT TO NORMALIZE
X = np.true_divide(X, 255)
X_test = np.true_divide(X_test, 255)

In [ ]:
# Some hand picked images for each class for plotting
selected_imgs = X[[1, 6, 5, 12, 26, 35, 62, 52, 46, 4]]

# <p style="padding:10px;background-color:#9c8497 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Train-val split</p>

In [ ]:
# Train-val split
split = 0.2
indices = np.arange(len(X))
np.random.shuffle(indices)
X_val = X[:int(split*X.shape[0])]
X_train = X[int(split*X.shape[0]):]
y_val = y[:int(split*X.shape[0])]
y_train = y[int(split*X.shape[0]):]
print("Number of training instances:\t", X.shape[0])
print("Number of validation instances:\t", y_val.shape[0])

# <p style="padding:10px;background-color:#9c8497 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Define some Function</p>

In [ ]:
from scipy.special import expit   # sigmoid

class RBM():
    def __init__(self, n_vis=28*28, n_hid=100):
        self.n_vis = n_vis
        self.n_hid = n_hid
        # Parameters
        self.W = 0.1 * np.random.randn(n_vis, n_hid)
        self.vbias = np.zeros(n_vis)
        self.hbias = -4.0 * np.ones(n_hid)
        # Gradients
        self.W_grad = np.zeros(self.W.shape)
        self.vbias_grad = np.zeros(n_vis)
        self.hbias_grad = np.zeros(n_hid)
        # Velocities - for momentum
        self.W_vel = np.zeros(self.W.shape)
        self.vbias_vel = np.zeros(n_vis)
        self.hbias_vel = np.zeros(n_hid)

    def h_given_v(self, v):
        '''
        input:
            - v: (batch_size, n_vis)
        output:
            - p(H|v) = sigmoid(W^Tv + hbias): (batch_size, n_hid)
            - samples from p(H|v): (batch_size, n_hid)
        '''
        p = expit(np.matmul(v, self.W) + self.hbias)
        return (p, np.random.binomial(1, p=p))

    def v_given_h(self, h):
        '''
        input:
            - h: (batch_size, n_hid)
        output:
            - p(V|h) = sigmoid(Wh + vbias): (batch_size, n_vis)
            - samples from p(V|h): (batch_size, n_vis)
        '''
        p = expit(np.matmul(h, self.W.T) + self.vbias)
        return (p, np.random.binomial(1, p=p))

    def compute_error_and_grads(self, batch, burn_in=0, num_steps=1, method="cd"):
        '''
        Function to compute the gradient of parameters and store in param_grad variables
        and reconstruction error.
        input:
            - batch: (batch_size, n_vis)
            - burn_in: Number of burn in steps for Gibbs sampling
            - num_steps: Number of steps for Gibbs sampling chain to run
            - method: Method for computing gradients. Available options:
                    - "cd": Contrastive Divergence
        output:
            - recon_error: Reconstruction error

        TODO:
            - Implement PCD and FPCD.
            - Use Gibbs sampling averaging, instead of taking just last value.
        '''
        b_size = batch.shape[0]
        v0 = batch.reshape(b_size, -1)

        # Compute gradients - Positive Phase
        ph0, h0 = self.h_given_v(v0)
        W_grad = np.matmul(v0.T, ph0)
        vbias_grad = np.sum(v0, axis=0)
        hbias_grad = np.sum(ph0, axis=0)

        # Compute gradients - Negative Phase

        # only contrastive with k = 1, i.e., method="cd"

        pv1, v1 = self.v_given_h(h0)
        ph1, h1 = self.h_given_v(pv1)

        W_grad -= np.matmul(pv1.T, ph1)
        vbias_grad -= np.sum(pv1, axis=0)
        hbias_grad -= np.sum(ph1, axis=0)

        self.W_grad = W_grad/b_size
        self.hbias_grad = hbias_grad/b_size
        self.vbias_grad = vbias_grad/b_size

        recon_err = np.mean(np.sum((v0 - pv1)**2, axis=1), axis=0) # sum of squared error averaged over the batch
        return recon_err

    def update_params(self, lr, momentum=0):
        '''
        Function to update the parameters based on the stored gradients.
        input:
            - lr: Learning rate
            - momentum
        '''
        self.W_vel *= momentum
        self.W_vel += (1.-momentum) * lr * self.W_grad
        self.W += self.W_vel

        self.vbias_vel *= momentum
        self.vbias_vel += (1.-momentum) * lr * self.vbias_grad
        self.vbias += self.vbias_vel

        self.hbias_vel *= momentum
        self.hbias_vel += (1.-momentum) * lr * self.hbias_grad
        self.hbias += self.hbias_vel

    def reconstruct(self, v):
        '''
        Reconstructing visible units from given v.
        v -> h0 -> v1
        input:
            - v: (batch_size, n_vis)
        output:
            - prob of reconstructed v: (batch_size, n_vis)
        '''
        ph0, h0 = self.h_given_v(v)
        pv1, v1 = self.v_given_h(ph0)
        return pv1

    def avg_free_energy(self, v):
        '''
        Compute the free energy of v averaged over the batch.
        input:
            - v: (batch_size, n_vis)
        output:
            - average of free energy: where free energy = - v.vbias - Sum_j (log(1 + exp(hbias + v_j*W_:,j)) )
        '''
        x = self.hbias + np.matmul(v, self.W)
        free_energy_batch = -np.matmul(v, self.vbias) - np.sum(np.log(1 + np.exp(x)), axis=1)
        return np.mean(free_energy_batch)

    def gen_model_sample(self, start=None, num_iters=1000):
        '''
        Generate random samples of visible unit from the model using Gibbs sampling.
        input:
            - start: Any starting value of v.
            - num_iters: Number of iterations of Gibbs sampling.
        '''
        if(start is None):
            v = np.random.randn(self.n_vis)
        else:
            v = start
        for _ in range(num_iters):
            ph, h = rbm.h_given_v(v)
            pv, v = rbm.v_given_h(h)
        return v

In [ ]:
def get_batches(data, batch_size, shuffle=False):
    '''
    Function to provide data in batches.
    input:
        - data: The data to be batched, each sample in one row.
        - batch_size: Size of one batch (last batch might be smaller)
        - shuffle: True if data should be shuffled.
    '''
    if(shuffle):
        np.random.shuffle(data)
    if(batch_size == -1):
        batch_size = len(data)
    num_batches = math.ceil(data.shape[0]/batch_size)
    for batch_num in range(num_batches):
        yield data[batch_num*batch_size:(batch_num+1)*batch_size]

In [ ]:
def plot_images(rbm, images, title='Reconstructed Images', save_as="Reconstructed Images"):
    '''
    Plot the images and their reconstruction from RBM.
    input:
        - rbm: The RBM object to be used.
        - images: The images to be reconstructed (batch, 28, 28)
    '''
    num_samples = len(images)
    plt.clf()
    fig, axes = plt.subplots(2, num_samples, gridspec_kw = {'wspace':0, 'hspace':0.1}, figsize=(2*num_samples, 4))
    fig.suptitle(title)
    text = axes[0, 0].text(-7, 14,"original", size=15,
                           verticalalignment='center', rotation=-270)
    text = axes[1, 0].text(-7, 13,"reconstructed", size=15,
                           verticalalignment='center', rotation=-270)
    for n in range(num_samples):
        axes[0, n].imshow(images[n].reshape(28, 28), cmap='gray')
        axes[1, n].imshow(rbm.reconstruct(images[n].reshape(28*28)).reshape(28, 28), cmap='gray')
        axes[0, n].axis('off')
        axes[1, n].axis('off')

    plt.savefig(save_as)
    plt.show()
    plt.close()

In [ ]:
def plot_weights(rbm, title='weights', save_as="weights"):
    '''
    Plot the weight parameter of the RBM, one for each hidden unit.
    '''
    plt.clf()
    fig, axes = plt.subplots(10, 10, gridspec_kw = {'wspace':0.1, 'hspace':0.1}, figsize=(8, 8))
    fig.suptitle(title)
    for i in range(10):
        for j in range(10):
            axes[i, j].imshow(rbm.W[:,i*10+j].reshape(28, 28), cmap='gray')
            axes[i, j].axis('off')

    plt.savefig(save_as)
    plt.show()
    plt.close()

# <p style="padding:10px;background-color:#9c8497 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">RBM Model</p>

In [ ]:
# Some parameters to set
batch_size = 50
num_epochs = 20
lr = 0.01
burn_in = 0
num_steps = 1

In [ ]:
# Our RBM object
rbm = RBM(n_vis=28*28, n_hid=100)

In [ ]:
# Training loop
errors = []
free_energies_val = []
free_energies_train = []
start_time = time()

# plot_images(rbm, selected_imgs,
#             title="Reconstructed Images    Epoch: 0",
#             save_as="recon_" + str(0))

for epoch in range(1, num_epochs+1):
    iteration = 0
    error = 0
    for batch in get_batches(X_train, batch_size, shuffle=True):
        iteration += 1

        # Compute gradients and errors
        error += rbm.compute_error_and_grads(batch, burn_in=burn_in, num_steps=num_steps)

        # Update parameters - use momentum as explained in Hinton's guide
        if(epoch > 5):
            rbm.update_params(lr, momentum=0.5)
        else:
            rbm.update_params(lr, momentum=0.9)

    #plot_images(rbm, selected_imgs,
    #        title="Reconstructed Images    Epoch: {}".format(epoch),
    #        save_as="recon_" + str(epoch))

    print("epoch:{} \t error:{:.4f} \t training time:{:.2f} s".format(epoch, error, time()-start_time))
    errors.append(error)

# <p style="padding:10px;background-color:#9c8497 ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Plots</p>

In [ ]:
plt.plot(errors)
plt.savefig("error_plot")

In [ ]:
plot_weights(rbm)

In [ ]:
plot_images(rbm, selected_imgs)